In [2]:
import numpy as np
import pandas as pd
import re
import ftplib

In [72]:
sample_featuretable = pd.read_csv('./sample_ref_featuretable.txt', sep='\t')
sample_featuretable = sample_featuretable[sample_featuretable["# feature"] == "CDS"]
print(sample_featuretable["product_accession"].tolist())
print(sample_featuretable.iloc[20:24])
sample_featuretable[sample_featuretable['product_accession'].apply(lambda x: isinstance(x, str)==False)]

['WP_010929569.1', 'WP_003806878.1', 'WP_003806879.1', 'WP_010929570.1', 'WP_010929571.1', nan, 'WP_003806883.1', 'WP_003806884.1', 'WP_003806885.1', 'WP_010929572.1', 'WP_003806887.1', 'WP_003806888.1', 'WP_003806889.1', 'WP_010929573.1', 'WP_003818292.1', 'WP_010929574.1', 'WP_010929575.1', nan, 'WP_010929576.1', nan, 'WP_003818297.1', 'WP_010929577.1', nan, 'WP_010929578.1', 'WP_019247164.1', 'WP_010929580.1', 'WP_019247165.1', 'WP_010929582.1', 'WP_010929583.1', nan, 'WP_003814398.1', 'WP_010929585.1', 'WP_010929586.1', 'WP_003814404.1', 'WP_003814405.1', 'WP_010929587.1', 'WP_019247766.1', 'WP_003814412.1', 'WP_023852754.1', nan, nan, 'WP_010929589.1', 'WP_003815265.1', 'WP_003815255.1', nan, 'WP_010929590.1', 'WP_003817748.1', 'WP_010929591.1', 'WP_029443813.1', nan, 'WP_010929592.1', 'WP_019247224.1', 'WP_003817737.1', 'WP_023852650.1', 'WP_010929594.1', 'WP_005012808.1', 'WP_023853472.1', 'WP_010929595.1', 'WP_010929596.1', 'WP_010929597.1', nan, 'WP_019248755.1', 'WP_010929599

,# feature,class,assembly,assembly_unit,seq_type,chromosome,genomic_accession,start,end,strand,product_accession,non-redundant_refseq,related_accession,name,symbol,GeneID,locus_tag,feature_interval_length,product_length,attributes
11,CDS,without_protein,GCF_002442695.1,Primary Assembly,chromosome,NaN,NZ_CP017162.1,5125,6126,+,NaN,NaN,NaN,asparaginase,NaN,NaN,BHR50_RS00030,1002,NaN,pseudo
43,CDS,without_protein,GCF_002442695.1,Primary Assembly,chromosome,NaN,NZ_CP017162.1,22008,24351,-,NaN,NaN,NaN,CHASE2 domain-containing protein,NaN,NaN,BHR50_RS00110,2344,NaN,pseudo
47,CDS,without_protein,GCF_002442695.1,Primary Assembly,chromosome,NaN,NZ_CP017162.1,25477,26213,-,NaN,NaN,NaN,response regulator transcription factor,NaN,NaN,BHR50_RS00120,737,NaN,pseudo
53,CDS,without_protein,GCF_002442695.1,Primary Assembly,chromosome,NaN,NZ_CP017162.1,28333,28650,-,NaN,NaN,NaN,CoA transferase,NaN,NaN,BHR50_RS00135,318,NaN,partial;pseudo
67,CDS,without_protein,GCF_002442695.1,Primary Assembly,chromosome,NaN,NZ_CP017162.1,34626,35575,+,NaN,NaN,NaN,IS481-like element IS481 family transposase,NaN,NaN,BHR50_RS00170,950,NaN,pseudo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7699,CDS,without_protein,GCF_002442695.1,Primary Assembly,chromosome,NaN,NZ_CP017162.1,4000364,4001040,+,NaN,NaN,NaN,imidazole glycerol phosphate synthase subunit ...,hisH,NaN,BHR50_RS19390,677,NaN,pseudo
7717,CDS,without_protein,GCF_002442695.1,Primary Assembly,chromosome,NaN,NZ_CP017162.1,4005412,4005591,+,NaN,NaN,NaN,helix-turn-helix transcriptional regulator,NaN,NaN,BHR50_RS20950,180,NaN,partial;pseudo
7719,CDS,without_protein,GCF_002442695.1,Primary Assembly,chromosome,NaN,NZ_CP017162.1,4005637,4005897,+,NaN,NaN,NaN,integrase core domain-containing protein,NaN,NaN,BHR50_RS20470,261,NaN,partial;pseudo
7771,CDS,without_protein,GCF_002442695.1,Primary Assembly,chromosome,NaN,NZ_CP017162.1,4028132,4028512,+,NaN,NaN,NaN,ATP-binding cassette domain-containing protein,NaN,NaN,BHR50_RS19560,381,NaN,partial;pseudo


In [42]:
sample_gff = pd.read_csv('./sample_ref_gff.gff', sep='\t', names=['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes'])
sample_genes = sample_gff[sample_gff['type'] == 'gene']
sample_cds = sample_gff[sample_gff['type'] == 'CDS']
print(sample_cds['attributes'].tolist()[0])
sample_cds

ID=cds-WP_010929569.1;Parent=gene-BHR50_RS00005;Dbxref=GenBank:WP_010929569.1;Name=WP_010929569.1;Ontology_term=GO:0002098,GO:0008033,GO:0050660;gbkey=CDS;gene=mnmG;go_function=flavin adenine dinucleotide binding|0050660||IEA;go_process=tRNA wobble uridine modification|0002098||IEA,tRNA processing|0008033||IEA;inference=COORDINATES: similar to AA sequence:RefSeq:WP_014905376.1;locus_tag=BHR50_RS00005;product=tRNA uridine-5-carboxymethylaminomethyl(34) synthesis enzyme MnmG;protein_id=WP_010929569.1;transl_table=11


,seqid,source,type,start,end,score,strand,phase,attributes
2,NZ_CP017162.1,Protein Homology,CDS,1.0,1920.0,.,+,0,ID=cds-WP_010929569.1;Parent=gene-BHR50_RS0000...
4,NZ_CP017162.1,Protein Homology,CDS,1917.0,2609.0,.,+,0,ID=cds-WP_003806878.1;Parent=gene-BHR50_RS0001...
6,NZ_CP017162.1,Protein Homology,CDS,2606.0,3403.0,.,+,0,ID=cds-WP_003806879.1;Parent=gene-BHR50_RS0001...
8,NZ_CP017162.1,Protein Homology,CDS,3410.0,4171.0,.,+,0,ID=cds-WP_010929570.1;Parent=gene-BHR50_RS0002...
10,NZ_CP017162.1,Protein Homology,CDS,4215.0,5132.0,.,+,0,ID=cds-WP_010929571.1;Parent=gene-BHR50_RS0002...
...,...,...,...,...,...,...,...,...,...
7971,NZ_CP017162.1,Protein Homology,CDS,4099705.0,4100709.0,.,+,0,ID=cds-WP_010931690.1;Parent=gene-BHR50_RS1987...
7973,NZ_CP017162.1,Protein Homology,CDS,4100814.0,4101251.0,.,+,0,ID=cds-WP_033446144.1;Parent=gene-BHR50_RS1988...
7975,NZ_CP017162.1,Protein Homology,CDS,4101248.0,4102771.0,.,+,0,ID=cds-WP_010931692.1;Parent=gene-BHR50_RS1988...
7977,NZ_CP017162.1,Protein Homology,CDS,4102847.0,4104013.0,.,-,0,ID=cds-WP_010931693.1;Parent=gene-BHR50_RS1989...


In [30]:
df = pd.DataFrame(columns=["ID", "Name", "gbkey", "gene", "gene_biotype", "locus_tag", "old_locus_tag"])
for row in sample_genes.iterrows():
    row = row[1]["attributes"].split(';')
    if any("gene=" in stri for stri in row):
        df.loc[len(df)] = {
            "ID": row[0].split('=')[1],
            "Name": row[1].split('=')[1],
            "gbkey": row[2].split('=')[1],
            "gene": row[3].split('=')[1],
            "gene_biotype": row[4].split('=')[1],
            "locus_tag": row[5].split('=')[1],
            "old_locus_tag": row[6].split('=')[1]
        }
    elif any("old_locus_tag=" in stri for stri in row):
        df.loc[len(df)] = {
            "ID": row[0].split('=')[1],
            "Name": row[1].split('=')[1],
            "gbkey": row[2].split('=')[1],
            "gene": None,
            "gene_biotype": row[3].split('=')[1],
            "locus_tag": row[4].split('=')[1],
            "old_locus_tag": row[5].split('=')[1]
        }
    else:
        df.loc[len(df)] = {
            "ID": row[0].split('=')[1],
            "Name": row[1].split('=')[1],
            "gbkey": row[2].split('=')[1],
            "gene": None,
            "gene_biotype": row[3].split('=')[1],
            "locus_tag": row[4].split('=')[1],
            "old_locus_tag": None
        }
df

['ID=gene-BHR50_RS00015', 'Name=BHR50_RS00015', 'gbkey=Gene', 'gene_biotype=protein_coding', 'locus_tag=BHR50_RS00015', 'old_locus_tag=BHR50_00015']
['ID=gene-BHR50_RS00020', 'Name=BHR50_RS00020', 'gbkey=Gene', 'gene_biotype=protein_coding', 'locus_tag=BHR50_RS00020', 'old_locus_tag=BHR50_00020']
['ID=gene-BHR50_RS00025', 'Name=BHR50_RS00025', 'gbkey=Gene', 'gene_biotype=protein_coding', 'locus_tag=BHR50_RS00025', 'old_locus_tag=BHR50_00025']
['ID=gene-BHR50_RS00035', 'Name=BHR50_RS00035', 'gbkey=Gene', 'gene_biotype=tRNA', 'locus_tag=BHR50_RS00035', 'old_locus_tag=BHR50_00040']
['ID=gene-BHR50_RS00040', 'Name=BHR50_RS00040', 'gbkey=Gene', 'gene_biotype=tRNA', 'locus_tag=BHR50_RS00040', 'old_locus_tag=BHR50_00045']
['ID=gene-BHR50_RS00045', 'Name=BHR50_RS00045', 'gbkey=Gene', 'gene_biotype=tRNA', 'locus_tag=BHR50_RS00045', 'old_locus_tag=BHR50_00050']
['ID=gene-BHR50_RS00055', 'Name=BHR50_RS00055', 'gbkey=Gene', 'gene_biotype=tRNA', 'locus_tag=BHR50_RS00055', 'old_locus_tag=BHR50_00060

,ID,Name,gbkey,gene,gene_biotype,locus_tag,old_locus_tag
0,gene-BHR50_RS00005,mnmG,Gene,mnmG,protein_coding,BHR50_RS00005,BHR50_00005
1,gene-BHR50_RS00010,rsmG,Gene,rsmG,protein_coding,BHR50_RS00010,BHR50_00010
2,gene-BHR50_RS00015,BHR50_RS00015,Gene,None,protein_coding,BHR50_RS00015,BHR50_00015
3,gene-BHR50_RS00020,BHR50_RS00020,Gene,None,protein_coding,BHR50_RS00020,BHR50_00020
4,gene-BHR50_RS00025,BHR50_RS00025,Gene,None,protein_coding,BHR50_RS00025,BHR50_00025
...,...,...,...,...,...,...,...
3644,gene-BHR50_RS19875,BHR50_RS19875,Gene,None,protein_coding,BHR50_RS19875,BHR50_19595
3645,gene-BHR50_RS19880,BHR50_RS19880,Gene,None,protein_coding,BHR50_RS19880,BHR50_19600
3646,gene-BHR50_RS19885,BHR50_RS19885,Gene,None,protein_coding,BHR50_RS19885,BHR50_19605
3647,gene-BHR50_RS19890,BHR50_RS19890,Gene,None,protein_coding,BHR50_RS19890,BHR50_19610


In [36]:
len(df[df["Name"].str.startswith("BHR") == True]["Name"].tolist())

2748

In [11]:
refid_df = pd.read_csv(genome_id_file, sep="\t")
refid_df = refid_df[refid_df["Assembly Accession"].str.contains("GCF_")]
print(refid_df.head())
genome_ids = refid_df["Assembly Accession"].tolist()

  Assembly Accession Assembly Name  \
0    GCF_000006945.2      ASM694v2   
1    GCF_000195955.2    ASM19595v2   
2    GCF_000009045.1      ASM904v1   
3    GCF_009035845.1   ASM903584v1   
4    GCF_000015785.2     ASM1578v2   

                                       Organism Name  \
0  Salmonella enterica subsp. enterica serovar Ty...   
1                   Mycobacterium tuberculosis H37Rv   
2         Bacillus subtilis subsp. subtilis str. 168   
3                            Acinetobacter baumannii   
4                          Bacillus velezensis FZB42   

   Annotation Count Gene Total  
0                         4717  
1                         4008  
2                         4536  
3                         3832  
4                         3882  


In [ ]:
    def getGreatestLastDigit(array: list, version: int):

        s = f"^.*((v{version})|(\.{version})).*$"
        l = list(filter(re.compile(s).match, array))
        print("\tfiltered: ", l)
        if len(l) == 1:
            return l[0]
        else:
            return l[max(range(len(l)), key=lambda x: int(re.search("^GCF_\\d+\\.(\\d)_.+$", l[x]).group(1)))]

    def getFTPGenes(accession):
        dir = f"{accession[4:7]}/{accession[7:10]}/{accession[10:13]}" # GCF_000195955.2_ASM19595v2/GCF_000195955.2_ASM19595v2_genomic.gtf.gz
        ftp.cwd(dir)
        print(dir, ftp.nlst())
        subdir = getGreatestLastDigit(ftp.nlst(), accession[-1]) if len(ftp.nlst()) != 1 else ftp.nlst()[0]
        ftp.cwd(subdir)
        file = list(filter(re.compile("^.+_feature_table\.txt\.gz$").match, ftp.nlst()))[0]

        url = f"https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/{dir}/{subdir}/{file}"

        with urllib.request.urlopen(url) as response:
            content = response.read()

        content = gzip.decompress(content)
        content = content[content.find(b"\nNC_")+1:-4]
        genome_df = pd.read_csv(BytesIO(content), sep="\t")
        # names=["feature", "class", "assembly", "assembly_unit", "seq_type", "chromosome", "genomic_accession", "start", "end", "strand", "product_accession", "non-redundant_refseq", "related_accession", "name", "symbol", "GeneID", "locus_tag", "feature_interval_length", "product_length", "attributes"]
        genome_df = genome_df[genome_df["# feature"]=="gene"]
        gene_symbols = genome_df["symbol"].tolist()
        gene_symbols = [x if type(x) == str else str(x) for x in gene_symbols]
        gene_ids = genome_df["GeneID"].tolist()

        # genes = []
        # for idx, row in genome_df.iterrows():
        #     try:
        #         genes.append(re.search(";Name=([a-zA-Z]+);gbkey", row["description"]).group(1))
        #     except:
        #         pass

        ftp.cwd("../../../..")
        return gene_symbols, gene_ids